In [ ]:
#Fig.9#
"""
Group-difference + Spearman correlation heatmaps (Case2 vs Case0)
Export as 5 PNGs (legend + 4 panels) so you can assemble in PPT,
with UNIFORM cell width/height across panels and no text clipping.

What this script does:
1) Merge basin-level metrics (NSE/KGE) with CAMELS attributes (*.txt, semicolon-separated).
2) Define two groups:
   - Group 1: Case2 outperforms Case0 (ΔNSE > 0)   [primary grouping]
   - Group 2: otherwise
3) Compute:
   - % difference between Group 1 and Group 2 for each attribute:
       pct_diff = 100 * (mean_g1 - mean_g2) / mean_g2
   - Skill scores (relative improvement) for NSE and KGE:
       SS = (M_case2 - M_case0) / (1 - M_case0 + eps)
   - Spearman correlation within each group between attribute and SS_NSE / SS_KGE
4) Select attributes to display (parsimony rule):
   Keep attribute if (|pct_diff| > 10) OR (max |rho| >= 0.2 across the 4 corr cells)
5) Export:
   - legend_colorbars.png
   - panel_a.png, panel_b.png, panel_c.png, panel_d.png
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import FuncFormatter

# ===================== Font: Times New Roman (Windows Fonts) =====================
import matplotlib as mpl
from matplotlib import font_manager as fm

FONT_DIR = Path("/mnt/c/Windows/Fonts")  # Times New Roman

def setup_times_new_roman(font_dir: Path):
    if not font_dir.exists():
        raise FileNotFoundError(f"FONT_DIR not found: {font_dir}")

    candidates = [
        font_dir / "times.ttf",
        font_dir / "timesbd.ttf",
        font_dir / "timesi.ttf",
        font_dir / "timesbi.ttf",
    ]
    existing = [p for p in candidates if p.exists()]

    if not existing:
        fallback = []
        for ext in ("*.ttf", "*.otf", "*.ttc"):
            fallback.extend(list(font_dir.glob(ext)))
        fallback = [p for p in fallback if "times" in p.name.lower()]
        existing = fallback

    if not existing:
        raise RuntimeError(f"No Times New Roman font files found under: {font_dir}")

    for fp in existing:
        try:
            fm.fontManager.addfont(str(fp))
        except Exception as e:
            print(f"[WARN] failed to add font {fp}: {e}")

    regular = candidates[0] if candidates[0].exists() else existing[0]
    tnr_prop = fm.FontProperties(fname=str(regular))
    tnr_name = tnr_prop.get_name()

    mpl.rcParams["font.family"] = tnr_name
    mpl.rcParams["font.sans-serif"] = [tnr_name]
    mpl.rcParams["axes.unicode_minus"] = False

    mpl.rcParams["mathtext.fontset"] = "custom"
    mpl.rcParams["mathtext.rm"] = tnr_name
    mpl.rcParams["mathtext.it"] = f"{tnr_name}:italic"
    mpl.rcParams["mathtext.bf"] = f"{tnr_name}:bold"

    print(f"[INFO] Using font: {tnr_name}")
    print(f"[INFO] Registered font files: {[p.name for p in existing]}")
    return tnr_prop, tnr_name

TNR_PROP, TNR_NAME = setup_times_new_roman(FONT_DIR)

# ===================== 1) CONFIG (EDIT ONLY HERE) =====================

BASE_DIR = Path("/mnt/d/desktop/paper_data/01")

CASE0_CSV = BASE_DIR / "model_data/case0/val_metrics_lead0.csv"
CASE2_CSV = BASE_DIR / "model_data/case3/val_metrics_lead0.csv"

ATTR_DIR = BASE_DIR / "static_attri"

# Files and their panel titles (order matters: first 4 will be exported as a–d)
PANEL_FILES = [
    ("camels_hydro.txt", "a. Hydrologic signatures"),
    ("camels_clim.txt",  "b. Climate"),
    ("camels_soil.txt",  "c. Soil"),
    ("camels_topo.txt",  "d. Topographic"),
    ("camels_vege.txt",  "e. Vegetation"),   # optional; we will compute table but not export panel by default
]

OUT_DIR = BASE_DIR / "paper_output" / "group_diff_corr_case3_vs_case0"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Grouping rule (primary):
# Group 1: ΔKGE > 0, Group 2: ΔKGE <= 0
GROUP_ON_METRIC = "NSE"

# skill score eps
EPS = 1e-6

# parsimony thresholds
PCT_DIFF_TH = 10.0
ABS_RHO_TH  = 0.2

# shared colorbar ranges
PCT_VMIN, PCT_VMAX = -30.0, 30.0
RHO_VMIN, RHO_VMAX = -0.4,  0.4

# for stable basin id join
BASIN_ZFILL = 8

# ---- Plot sizing controls (uniform across panels)
# cell size in inches; increase to make everything less crowded
CELL_W_IN = 0.55
CELL_H_IN = 0.30

# margins in inches (increase left margin if y-labels are long)
LEFT_MARGIN_IN   = 3.6
RIGHT_MARGIN_IN  = 0.4
TOP_MARGIN_IN    = 0.7
BOTTOM_MARGIN_IN = 0.9

# font sizes
TITLE_FS = 16
LABEL_FS = 12
TICK_FS  = 12
NUM_FS   = 12

LEGEND_LABEL_FS = 20   # colorbar 
LEGEND_TICK_FS  = 20   # colorbar 


# ===================== Times New Roman =====================
plt.rcParams.update({
    "font.family": TNR_NAME,
    "font.size": 12,
    "axes.unicode_minus": False,
})

# ===================== 2) HELPERS =====================

def _std_basin_id_str(s: pd.Series, zfill: int = BASIN_ZFILL) -> pd.Series:
    return (
        s.astype(str).str.strip()
        .str.replace(r"\.0$", "", regex=True)
        .str.replace(r"[^0-9A-Za-z]", "", regex=True)
        .str.zfill(zfill)
    )

def _find_id_col(cols):
    priority = ["basin_id", "gauge_id", "gage_id", "gaging_id", "site_id", "hru_id", "id"]
    lower = {str(c).strip().lower(): c for c in cols}
    for key in priority:
        if key in lower:
            return lower[key]
    for c in cols:
        cl = str(c).strip().lower()
        if "id" in cl and any(k in cl for k in ["gauge", "basin", "gage", "gaging", "hru", "site"]):
            return c
    return None

def read_metrics(fp: Path) -> pd.DataFrame:
    df = pd.read_csv(fp)
    if "basin_id" not in df.columns:
        raise RuntimeError(f"{fp} missing 'basin_id'. columns={list(df.columns)[:20]}")
    for m in ["NSE", "KGE"]:
        if m not in df.columns:
            raise RuntimeError(f"{fp} missing metric '{m}'. columns={list(df.columns)[:20]}")
    df["basin_id_str"] = _std_basin_id_str(df["basin_id"])
    df["NSE"] = pd.to_numeric(df["NSE"], errors="coerce")
    df["KGE"] = pd.to_numeric(df["KGE"], errors="coerce")
    return df[["basin_id_str", "NSE", "KGE"]].copy()

def skill_score(case: pd.Series, base: pd.Series, eps: float = EPS) -> pd.Series:
    case = pd.to_numeric(case, errors="coerce")
    base = pd.to_numeric(base, errors="coerce")
    denom = 1.0 - base
    denom = denom.where(np.abs(denom) >= eps, np.sign(denom).replace(0, 1.0) * eps)
    return (case - base) / denom

def read_attr_semicolon(fp: Path) -> pd.DataFrame:
    df = pd.read_csv(fp, sep=";", engine="python", comment="#")
    df.columns = [str(c).strip() for c in df.columns]
    id_col = _find_id_col(df.columns)
    if id_col is None:
        raise RuntimeError(f"Cannot find id column in {fp.name}. columns head={list(df.columns)[:30]}")
    df["basin_id_str"] = _std_basin_id_str(df[id_col])
    for c in df.columns:
        if c in [id_col, "basin_id_str"]:
            continue
        df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def spearman_rho(x: pd.Series, y: pd.Series) -> float:
    m = np.isfinite(x.values) & np.isfinite(y.values)
    if m.sum() < 10:
        return np.nan
    return pd.Series(x.values[m]).corr(pd.Series(y.values[m]), method="spearman")

def pct_diff(mean_g1: float, mean_g2: float) -> float:
    if not np.isfinite(mean_g1) or not np.isfinite(mean_g2) or mean_g2 == 0:
        return np.nan
    return 100.0 * (mean_g1 - mean_g2) / mean_g2

def build_div_cmap():
    return LinearSegmentedColormap.from_list(
        "bwr_soft",
        ["#2b6cb0", "#7fb3d5", "#f7f7f7", "#f5b7b1", "#c0392b"],
        N=256
    )

# ===================== 3) CORE COMPUTATION PER PANEL =====================

def panel_stats(df_join: pd.DataFrame, attr_cols: list):
    g1 = df_join[df_join["group"] == 1]
    g2 = df_join[df_join["group"] == 2]

    rows = []
    for a in attr_cols:
        x1 = g1[a]
        x2 = g2[a]

        m1 = float(np.nanmean(x1.values)) if x1.notna().any() else np.nan
        m2 = float(np.nanmean(x2.values)) if x2.notna().any() else np.nan
        pdiff = pct_diff(m1, m2)

        rho11 = spearman_rho(x1, g1["SS_NSE"])
        rho12 = spearman_rho(x1, g1["SS_KGE"])
        rho21 = spearman_rho(x2, g2["SS_NSE"])
        rho22 = spearman_rho(x2, g2["SS_KGE"])

        rows.append((a, pdiff, rho11, rho12, rho21, rho22))

    out = pd.DataFrame(
        rows,
        columns=["attr", "pct_diff", "rho_g1_nse", "rho_g1_kge", "rho_g2_nse", "rho_g2_kge"]
    )

    out["max_abs_rho"] = out[["rho_g1_nse","rho_g1_kge","rho_g2_nse","rho_g2_kge"]].abs().max(axis=1)
    keep = (out["pct_diff"].abs() > PCT_DIFF_TH) | (out["max_abs_rho"] >= ABS_RHO_TH)
    out = out.loc[keep].copy()

    out["score"] = out["pct_diff"].abs().fillna(0) / 10.0 + out["max_abs_rho"].fillna(0)
    out = out.sort_values("score", ascending=False).drop(columns=["score"])

    names = out["attr"].tolist()
    pct = out[["pct_diff"]].to_numpy(dtype=float)
    rho_g1 = out[["rho_g1_nse","rho_g1_kge"]].to_numpy(dtype=float)
    rho_g2 = out[["rho_g2_nse","rho_g2_kge"]].to_numpy(dtype=float)
    return names, pct, rho_g1, rho_g2, out

# ===================== 4) PLOTTING (EXPORT LEGEND + SINGLE PANEL) =====================

def _panel_to_full_matrix(pct, rho_g1, rho_g2,
                          pct_vmin, pct_vmax, rho_vmin, rho_vmax):
    n = pct.shape[0]
    mat = np.full((n, 5), np.nan, dtype=float)
    mat[:, 0] = np.clip(pct[:, 0], pct_vmin, pct_vmax)
    mat[:, 1:3] = np.clip(rho_g1, rho_vmin, rho_vmax)
    mat[:, 3:5] = np.clip(rho_g2, rho_vmin, rho_vmax)
    return mat

def plot_legends(out_png: Path, cmap,
                 pct_vmin=PCT_VMIN, pct_vmax=PCT_VMAX,
                 rho_vmin=RHO_VMIN, rho_vmax=RHO_VMAX,
                 dpi=300,
                 legend_label_fs=18,     
                 legend_tick_fs=14):   
    fig = plt.figure(figsize=(8.5, 2.2), dpi=dpi)
    ax = fig.add_subplot(111)
    ax.axis("off")

    # Percentage difference colorbar
    cb1_ax = fig.add_axes([0.12, 0.9, 0.76, 0.18])
    sm1 = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(pct_vmin, pct_vmax))
    cbar1 = fig.colorbar(sm1, cax=cb1_ax, orientation="horizontal")
    cbar1.set_label("Percentage difference between Group 1 and Group 2", fontsize=legend_label_fs)
    cbar1.ax.tick_params(labelsize=legend_tick_fs)
    cbar1.ax.xaxis.set_label_position("top")
    cbar1.ax.xaxis.label.set_horizontalalignment("center")
    cbar1.set_ticks(np.linspace(pct_vmin, pct_vmax, 7))
    cbar1.ax.xaxis.set_major_formatter(
        FuncFormatter(lambda x, pos: f"{int(x)}%")
    )
     

    # Spearman correlation colorbar
    cb2_ax = fig.add_axes([0.12, 0.18, 0.76, 0.18])
    sm2 = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(rho_vmin, rho_vmax))
    cbar2 = fig.colorbar(sm2, cax=cb2_ax, orientation="horizontal")
    cbar2.set_label("Spearman correlation between skill scores and \nwatershed characteristics", fontsize=legend_label_fs)
    cbar2.ax.xaxis.set_label_position("top")
    cbar2.ax.xaxis.label.set_horizontalalignment("center")
    cbar2.set_ticks(np.linspace(rho_vmin, rho_vmax, 9))
    cbar2.ax.tick_params(labelsize=legend_tick_fs)   
    fig.savefig(out_png, dpi=dpi, bbox_inches="tight")
    plt.close(fig)
    print(f"Saved: {out_png}")


def plot_single_panel(pr: dict,
                      out_png: Path,
                      cmap,
                      pct_vmin=PCT_VMIN, pct_vmax=PCT_VMAX,
                      rho_vmin=RHO_VMIN, rho_vmax=RHO_VMAX,
                      cell_w_in=CELL_W_IN, cell_h_in=CELL_H_IN,
                      left_margin_in=LEFT_MARGIN_IN, right_margin_in=RIGHT_MARGIN_IN,
                      top_margin_in=TOP_MARGIN_IN, bottom_margin_in=BOTTOM_MARGIN_IN,
                      title_fs=TITLE_FS, label_fs=LABEL_FS, tick_fs=TICK_FS, num_fs=NUM_FS,
                      gap1=0.25, gap2=0.25,
                      dpi=300):
    title = pr["title"]
    names = pr["names"]
    pct = pr["pct"]
    rho_g1 = pr["rho_g1"]
    rho_g2 = pr["rho_g2"]

    n = len(names)
    if n == 0:
        fig = plt.figure(figsize=(8, 3), dpi=dpi)
        ax = fig.add_subplot(111)
        ax.axis("off")
        ax.set_title(title, loc="left", fontsize=title_fs, fontweight="bold")
        ax.text(0.5, 0.5, "No attributes passed the filter.", ha="center", va="center")
        fig.savefig(out_png, dpi=dpi, bbox_inches="tight")
        plt.close(fig)
        print(f"Saved: {out_png}")
        return

    x0 = 0.0
    x_pct0, x_pct1 = x0, x0 + 1.0
    x_g10 = x_pct1 + gap1
    x_g11 = x_g10 + 2.0
    x_g20 = x_g11 + gap2
    x_g21 = x_g20 + 2.0
    x_max = x_g21

    grid_w_in = x_max * cell_w_in
    grid_h_in = n * cell_h_in
    fig_w = left_margin_in + grid_w_in + right_margin_in
    fig_h = top_margin_in + grid_h_in + bottom_margin_in

    fig = plt.figure(figsize=(fig_w, fig_h), dpi=dpi)
    ax = fig.add_axes([
        left_margin_in / fig_w,
        bottom_margin_in / fig_h,
        grid_w_in / fig_w,
        grid_h_in / fig_h
    ])

    mat_pct = np.clip(pct, pct_vmin, pct_vmax)
    mat_g1  = np.clip(rho_g1, rho_vmin, rho_vmax)
    mat_g2  = np.clip(rho_g2, rho_vmin, rho_vmax)

    ax.imshow(mat_pct, cmap=cmap, vmin=pct_vmin, vmax=pct_vmax,
              aspect="auto", origin="upper", extent=(x_pct0, x_pct1, n, 0))
    ax.imshow(mat_g1, cmap=cmap, vmin=rho_vmin, vmax=rho_vmax,
              aspect="auto", origin="upper", extent=(x_g10, x_g11, n, 0))
    ax.imshow(mat_g2, cmap=cmap, vmin=rho_vmin, vmax=rho_vmax,
              aspect="auto", origin="upper", extent=(x_g20, x_g21, n, 0))

    ax.set_xlim(x0, x_max)
    ax.set_ylim(n, 0)

    ax.set_yticks(np.arange(n) + 0.5)
    ax.set_yticklabels(names, fontsize=tick_fs)
    ax.tick_params(axis="y", length=0)

    xt = [x_pct0 + 0.5]
    xt += [x_g10 + 0.5, x_g10 + 1.5]
    xt += [x_g20 + 0.5, x_g20 + 1.5]
    ax.set_xticks(xt)
    ax.set_xticklabels(
        ["Percentage\ndifference (%)", "NSE$_{SS}$", "KGE$_{SS}$", "NSE$_{SS}$", "KGE$_{SS}$"],
        fontsize=tick_fs
    )
    ax.tick_params(axis="x", length=0)

    ax.text((x_g10 + x_g11) / 2, -0.35, "Group 1",
            ha="center", va="center", fontsize=label_fs, fontstyle="italic",
            transform=ax.transData, clip_on=False)
    ax.text((x_g20 + x_g21) / 2, -0.35, "Group 2",
            ha="center", va="center", fontsize=label_fs, fontstyle="italic",
            transform=ax.transData, clip_on=False)

    for y in range(n + 1):
        ax.plot([x_pct0, x_pct1], [y, y], color="k", linewidth=0.8, alpha=1)
        ax.plot([x_g10,  x_g11],  [y, y], color="k", linewidth=0.8, alpha=1)
        ax.plot([x_g20,  x_g21],  [y, y], color="k", linewidth=0.8, alpha=1)

    for xx in [x_pct0, x_pct1, x_g10, x_g11, x_g20, x_g21]:
        ax.plot([xx, xx], [0, n], color="k", linewidth=0.8)

    ax.plot([x_g10 + 1.0, x_g10 + 1.0], [0, n], color="k", linewidth=0.8, alpha=1)
    ax.plot([x_g20 + 1.0, x_g20 + 1.0], [0, n], color="k", linewidth=0.8, alpha=1)

    for i in range(n):
        v = mat_pct[i, 0]
        if np.isfinite(v):
            ax.text(x_pct0 + 0.5, i + 0.5, f"{v:.2f}", ha="center", va="center", fontsize=num_fs)

        for j in range(2):
            v = mat_g1[i, j]
            if np.isfinite(v):
                ax.text(x_g10 + (j + 0.5), i + 0.5, f"{v:.2f}", ha="center", va="center", fontsize=num_fs)

        for j in range(2):
            v = mat_g2[i, j]
            if np.isfinite(v):
                ax.text(x_g20 + (j + 0.5), i + 0.5, f"{v:.2f}", ha="center", va="center", fontsize=num_fs)

    for s in ax.spines.values():
        s.set_visible(False)

    fig.savefig(out_png, dpi=dpi, bbox_inches="tight")
    plt.close(fig)
    print(f"Saved: {out_png}")

# ===================== 5) MAIN =====================

def main():
    d0 = read_metrics(CASE0_CSV).rename(columns={"NSE": "NSE_0", "KGE": "KGE_0"})
    d2 = read_metrics(CASE2_CSV).rename(columns={"NSE": "NSE_2", "KGE": "KGE_2"})
    dm = d0.merge(d2, on="basin_id_str", how="inner")
    print(f"[INFO] Common basins (Case0 & Case2): {len(dm)}")

    dm["SS_NSE"] = skill_score(dm["NSE_2"], dm["NSE_0"])
    dm["SS_KGE"] = skill_score(dm["KGE_2"], dm["KGE_0"])
    dm["dNSE"] = dm["NSE_2"] - dm["NSE_0"]
    dm["dKGE"] = dm["KGE_2"] - dm["KGE_0"]

    if GROUP_ON_METRIC.upper() == "NSE":
        dm["group"] = (dm["dNSE"] > 0).astype(int).replace({0: 2, 1: 1})
    elif GROUP_ON_METRIC.upper() == "KGE":
        dm["group"] = (dm["dKGE"] > 0).astype(int).replace({0: 2, 1: 1})
    else:
        raise ValueError("GROUP_ON_METRIC must be 'NSE' or 'KGE'.")

    n1 = (dm["group"] == 1).sum()
    n2 = (dm["group"] == 2).sum()
    print(f"[INFO] Group1 (improved): {n1}, Group2 (not improved): {n2}")

    panel_results = []
    for fname, title in PANEL_FILES:
        fp = ATTR_DIR / fname
        df_attr = read_attr_semicolon(fp)

        skip_cols = {"basin_id_str"}
        id_col = _find_id_col(df_attr.columns)
        if id_col is not None:
            skip_cols.add(id_col)

        attr_cols = [c for c in df_attr.columns if c not in skip_cols and pd.api.types.is_numeric_dtype(df_attr[c])]

        df_join = dm.merge(df_attr[["basin_id_str"] + attr_cols], on="basin_id_str", how="inner")
        print(f"[INFO] {fname}: merged rows={len(df_join)}, attr_cols={len(attr_cols)}")

        names, pct, rho_g1, rho_g2, out_tbl = panel_stats(df_join, attr_cols)

        panel_results.append({
            "title": title,
            "names": names,
            "pct": pct,
            "rho_g1": rho_g1,
            "rho_g2": rho_g2,
        })

    cmap = build_div_cmap()

    legend_png = OUT_DIR / "legend_colorbars.png"
    plot_legends(
        legend_png, cmap,
        pct_vmin=PCT_VMIN, pct_vmax=PCT_VMAX,
        rho_vmin=RHO_VMIN, rho_vmax=RHO_VMAX,
        dpi=300,
        legend_label_fs=LEGEND_LABEL_FS,
        legend_tick_fs=LEGEND_TICK_FS
    )


    panel_tags = ["a", "b", "c", "d"]
    for tag, pr in zip(panel_tags, panel_results[:4]):
        out_png = OUT_DIR / f"panel_{tag}.png"
        plot_single_panel(
            pr, out_png, cmap,
            pct_vmin=PCT_VMIN, pct_vmax=PCT_VMAX,
            rho_vmin=RHO_VMIN, rho_vmax=RHO_VMAX,
            cell_w_in=CELL_W_IN, cell_h_in=CELL_H_IN,
            left_margin_in=LEFT_MARGIN_IN, right_margin_in=RIGHT_MARGIN_IN,
            top_margin_in=TOP_MARGIN_IN, bottom_margin_in=BOTTOM_MARGIN_IN,
            title_fs=TITLE_FS, label_fs=LABEL_FS, tick_fs=TICK_FS, num_fs=NUM_FS,
            dpi=300
        )

    print("[DONE] Exported: legend_colorbars.png + panel_a/b/c/d.png")
    print(f"[OUT_DIR] {OUT_DIR}")

if __name__ == "__main__":
    main()


[INFO] Using font: Times New Roman
[INFO] Registered font files: ['times.ttf', 'timesbd.ttf', 'timesi.ttf', 'timesbi.ttf']
[INFO] Common basins (Case0 & Case2): 531
[INFO] Group1 (improved): 286, Group2 (not improved): 245
[INFO] camels_hydro.txt: merged rows=531, attr_cols=13
[INFO] camels_clim.txt: merged rows=531, attr_cols=11
[INFO] camels_soil.txt: merged rows=531, attr_cols=11
[INFO] camels_topo.txt: merged rows=531, attr_cols=6
[INFO] camels_vege.txt: merged rows=531, attr_cols=9


findfont: Font family ['cursive'] not found. Falling back to DejaVu Sans.
findfont: Generic family 'cursive' not found because none of the following families were found: Apple Chancery, Textile, Zapf Chancery, Sand, Script MT, Felipa, Comic Neue, Comic Sans MS, cursive


Saved: /mnt/d/desktop/paper_data/01/paper_output/group_diff_corr_case3_vs_case0/legend_colorbars.png
Saved: /mnt/d/desktop/paper_data/01/paper_output/group_diff_corr_case3_vs_case0/panel_a.png
Saved: /mnt/d/desktop/paper_data/01/paper_output/group_diff_corr_case3_vs_case0/panel_b.png
Saved: /mnt/d/desktop/paper_data/01/paper_output/group_diff_corr_case3_vs_case0/panel_c.png
Saved: /mnt/d/desktop/paper_data/01/paper_output/group_diff_corr_case3_vs_case0/panel_d.png
[DONE] Exported: legend_colorbars.png + panel_a/b/c/d.png
[OUT_DIR] /mnt/d/desktop/paper_data/01/paper_output/group_diff_corr_case3_vs_case0
